In [73]:
import pandas as pd
from pandas.api.types import is_string_dtype
import random
from matplotlib import rcParams
import matplotlib.pyplot as plt
import os
import time
import numpy as np
import glob

In [74]:
# define output file
Outputpath = 'D:/Powertac/finals-2021/data/'
Outputfilename = 'output' + '.csv'
Outputfilepath = Outputpath + Outputfilename

# define logtool class used to extract game data
logtool_class = 'MktPriceStats'

# find input .csv extracted by powertac logtools based on filename
path = 'D:/Powertac/finals-2021/data/'
data_filename = '*mktPr*'
#data_input = path + data_filename + '.csv'

# create dataframe that will be filled by iterated files
data = []
df_main = pd.DataFrame(data)

# search for all files containing string in selected path
if os.path.isdir(path) == True:
    matching_files = glob.glob(path + data_filename)

    # loop for iterating over all files of one datatype (e.g. Market Prices)
    for matching_file in matching_files:
        print(matching_file)

        if logtool_class == 'MktPriceStats':
            #df = pd.read_csv(data_input, header=None)
            df = pd.read_csv(matching_file, header=None)
            # create columns for cleared trades (CT)
            df.columns = ['Timeslot', 'Weekday', 'Hour', 'CT1', 'CT2', 'CT3', 'CT4', 'CT5', 'CT6', 'CT7', 'CT8',\
                          'CT9', 'CT10', 'CT11','CT12', 'CT13', 'CT14', 'CT15', 'CT16', 'CT17', 'CT18', 'CT19', \
                          'CT20','CT21', 'CT22', 'CT23', 'CT24']
            #df = df.set_index('Timeslot')
        else:
            print('Logtool Class not supported')

        # split cleared trades into cleared amounts (CA) and cleared prices (CP)
        c_index = 0
        for column in df.columns[2:]:
            if is_string_dtype(df[column]) == True:
                df[column] = df[column].str.strip('[]').astype(str)
                c_index = c_index + 1
                CA = 'CA' + str(c_index)
                CP = 'CP' + str(c_index)
                df[[CA,CP]] = df[column].str.split(' ', expand = True)
                df = df.drop(column, axis=1)


        #print(df.head())

        # create new Dataframe for the prediction input
        data = []
        df1 = pd.DataFrame(data)
        #df1.index.names = ['Timeslot']
        df1 = pd.concat([df1, df.Timeslot], axis=1)
        df1 = pd.concat([df1, df.Weekday], axis=1)

        # fill with cleared trades
        for column in df.columns[2:]:
            df1 = pd.concat([df1, df[column]], axis=1)

        # add more features:
        # cleared trades on hour before
        for column in df.columns[2:]:
            name = column + '_H-1'
            last_column = len(df1.columns)
            df1.insert(len(df1.columns), name, df[column].shift(1))

        # cleared trades on day before
        for column in df.columns[2:]:
            name = column + '_D-1'
            last_column = len(df1.columns)
            df1 = pd.concat([df1, df[column].shift(24).rename(name)], axis=1)  

        # cleared trades on week before
        for column in df.columns[2:]:
            name = column + '_W-1'
            last_column = len(df1.columns)
            df1 = pd.concat([df1, df[column].shift(168).rename(name)], axis=1)

        df1 = df1.fillna(0)  
        #df1

        #df_main = pd.concat([df1])
        df_main = pd.concat([df_main, df1])
        print(df_main)

    # create output file from dataframe
    df_main.to_csv(Outputfilepath)
else:
    print('Path not existing.')

D:/Powertac/finals-2021/data\mktPr1.csv
      Timeslot  Weekday  Hour      CA1      CP1     CA2      CP2     CA3  \
0          363        4     3  14.1679  32.8618     0.0      0.0     0.0   
1          364        4     4    5.118  13.5212  1.6868   5.5907     0.0   
2          365        4     5   1.6493   2.4414  3.4603   6.3035  1.5181   
3          366        4     6   2.7739   4.0663  1.4844   0.4172  3.1143   
4          367        4     7   2.6573   3.5092  2.4965  37.6041   1.336   
...        ...      ...   ...      ...      ...     ...      ...     ...   
1374      1737        5     9      0.0      0.0     0.0      0.0     0.0   
1375      1738        5    10      0.0      0.0     0.0      0.0     0.0   
1376      1739        5    11      0.0      0.0     0.0      0.0     0.0   
1377      1740        5    12      0.0      0.0     0.0      0.0     0.0   
1378      1741        5    13      0.0      0.0     0.0      0.0     0.0   

         CP3     CA4  ... CA20_W-1 CP20_W-1 CA2

      Timeslot  Weekday  Hour      CA1      CP1     CA2      CP2     CA3  \
0          363        4     3  14.1679  32.8618     0.0      0.0     0.0   
1          364        4     4    5.118  13.5212  1.6868   5.5907     0.0   
2          365        4     5   1.6493   2.4414  3.4603   6.3035  1.5181   
3          366        4     6   2.7739   4.0663  1.4844   0.4172  3.1143   
4          367        4     7   2.6573   3.5092  2.4965  37.6041   1.336   
...        ...      ...   ...      ...      ...     ...      ...     ...   
1628      1991        6    23      0.0      0.0     0.0      0.0     0.0   
1629      1992        7     0      0.0      0.0     0.0      0.0     0.0   
1630      1993        7     1      0.0      0.0     0.0      0.0     0.0   
1631      1994        7     2      0.0      0.0     0.0      0.0     0.0   
1632      1995        7     3      0.0      0.0     0.0      0.0     0.0   

         CP3     CA4  ... CA20_W-1 CP20_W-1 CA21_W-1 CP21_W-1 CA22_W-1  \
0        0.0 

      Timeslot  Weekday  Hour      CA1      CP1     CA2      CP2     CA3  \
0          363        4     3  14.1679  32.8618     0.0      0.0     0.0   
1          364        4     4    5.118  13.5212  1.6868   5.5907     0.0   
2          365        4     5   1.6493   2.4414  3.4603   6.3035  1.5181   
3          366        4     6   2.7739   4.0663  1.4844   0.4172  3.1143   
4          367        4     7   2.6573   3.5092  2.4965  37.6041   1.336   
...        ...      ...   ...      ...      ...     ...      ...     ...   
1345      1708        2     4      0.0      0.0     0.0      0.0     0.0   
1346      1709        2     5      0.0      0.0     0.0      0.0     0.0   
1347      1710        2     6      0.0      0.0     0.0      0.0     0.0   
1348      1711        2     7      0.0      0.0     0.0      0.0     0.0   
1349      1712        2     8      0.0      0.0     0.0      0.0     0.0   

         CP3     CA4  ... CA20_W-1 CP20_W-1 CA21_W-1 CP21_W-1 CA22_W-1  \
0        0.0 

In [7]:
#list(df_main.columns)